In [1]:
import os
os.chdir('../../src/models')
import simulate_pandemic as simp
from policies import policies, costs
from tqdm import tqdm
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import datetime

from policies import costs


color_map = {
    'Lockdown':          'rgb(0.83, 0.13, 0.15)',
    'Hard Quarantine':    'rgb(0.85, 0.35, 0.13)',
    'Light Quarantine':   'rgb(0.97, 0.91, 0.56)',
    'Social Distancing':  'rgb(0.67, 0.88, 0.69)',
    'Unrestricted':        'rgb(0.86, 0.86, 0.86)'    
}

Loading Graph...Done!


In [68]:
def make_SIR_graph(data, ):
    d_counts = pd.DataFrame([pd.Series(d).value_counts() for d in data])
    d_counts.fillna(0, inplace=True)

    fig = go.Figure()
    x = list(range(len(d_counts[0])+1))

    fig.add_trace(go.Scatter(x=x, y=d_counts[-1]/55e3, name='removed',line_color='green'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[0]/55e3, name='susceptible', line_color='blue'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[1]/55e3, name='exposed',line_color ='orange'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[2]/55e3, name='infected', line_color = 'red'))
    fig.add_trace(go.Scatter(x=x, y=d_counts[3]/55e3, name='hospitalized', line_color = 'purple'))
    fig.update_layout(hovermode='x')
    fig.show()
    
    return d_counts

def make_beds_graph(data, actions, step_size, title, color_map, make_df = True):
    fig = go.Figure()
    
    if make_df:
        data = pd.DataFrame([pd.Series(d).value_counts() for d in data])
        data.fillna(0, inplace=True)
    
    actions = list(map(color_map.get,  actions))

    x = list(range(len(data)+1))
    
    #\definecolor{royalblue(web)}{rgb}{0.25, 0.41, 0.88}
    fig.add_trace(go.Scatter(x=x, y=data[3]/55e3, name='hospitalized', line_color = 'rgb(0.25, 0.41, 0.88)',
                            line=dict(width=3.5),  yaxis="y2"))
    #\definecolor{firebrick}{rgb}{0.7, 0.13, 0.13}
    fig.add_trace(go.Scatter(x=x, y=data[1]/55e3, name='exposed', line_color = 'rgb(0.7, 0.13, 0.13)',
                            line=dict(width=4)))
    fig.add_trace(go.Scatter(x=x, y=len(data)*[0.0025], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2), yaxis="y2"))
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=x[step_size*i],
                y0=0,
                x1=x[step_size*(i+1)],
                y1=1,
                fillcolor=a,
                opacity=0.45,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)] 
    )

    for k,v in color_map.items():
        fig.add_trace(go.Bar(x=[None], y=[None], marker=dict(color=v), name = k))

    fig.update_layout(coloraxis = {'colorscale':'deep'}, xaxis={'showgrid': False,},
                      yaxis = {'showgrid': False, 'zerolinecolor': 'black',
                               'zerolinewidth': .5, 'zeroline': False,  'title': 'Proportion of Population'},
                      showlegend=True, hovermode="x")
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(0, len(data)+1, 14)),
            ticktext = list(range(0, int(len(data)/7) + 1, 2)),
            title = 'Weeks'
            )
        )
    fig.update_layout(showlegend=False)
    
    fig.update_layout(
        yaxis=dict(
            title="Proportion of Population Exposed",
            titlefont=dict(
                color="rgb(0.7, 0.13, 0.13)"
            ),
            tickfont=dict(
                color="rgb(0.7, 0.13, 0.13)"
            )
        ),
        yaxis2=dict(
            title="Proportion of Population Hospitalized",
            titlefont=dict(
                color="rgb(0.25, 0.41, 0.88)"
            ),
            tickfont=dict(
                color="rgb(0.25, 0.41, 0.88)"
            ),
            anchor="x",
            overlaying="y",
            side="right"
        ),)

    fig.write_image(f"{title}.pdf")


    fig.show()

In [84]:
def run_simulation(policy, step_size = 7, days = 280):
    pop_matrix, contacts_infected = simp.init_infection(.0001)

    data = []
    actions = []
   
    for day in tqdm(range(1, days+1)):
        #if less than 20% still susceptible, break simulation
        if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break
            
        # Choose a new policy at each week
        if day % step_size == 1:
            action = policy(day)
            restrictions = policies[action]
            actions.append(action)

        pop_matrix, contacts_infected = simp.spread_infection(pop_matrix, restrictions, day, contacts_infected)
        #pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix, key=lambda x: x[0]))[:, 1])

    return data, actions, contacts_infected, pop_matrix


In [89]:
def unrestricted(day):
    return 'Unrestricted'

def early_end(day):
    if day < 28:
        return 'Unrestricted'
    if day <= 112: 
        return 'Light Quarantine'
    return 'Social Distancing'
    
def intermitent_rest(day):
    actions = ['Social Distancing', 'Social Distancing', 'Light Quarantine', 'Light Quarantine']*50
    return actions[int(day/7)]

def random_pol(day):
    return np.random.choice(list(policies.keys()))

In [90]:
data, actions, contacts_infected, pop_matrix = run_simulation(early_end)


100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [01:08<00:00,  4.09it/s]


In [91]:
make_SIR_graph(data)

,-1.0,0.0,1.0,2.0,3.0
0,0.0,55487.0,5.0,0.0,0.0
1,0.0,55487.0,5.0,0.0,0.0
2,0.0,55487.0,5.0,0.0,0.0
3,0.0,55487.0,5.0,0.0,0.0
4,0.0,55487.0,5.0,0.0,0.0
...,...,...,...,...,...
275,25587.0,29528.0,116.0,230.0,31.0
276,25622.0,29510.0,113.0,216.0,31.0
277,25658.0,29489.0,119.0,197.0,29.0
278,25691.0,29469.0,111.0,191.0,30.0


In [92]:
make_beds_graph(data, actions, 7, 'intermitent', color_map)

In [5]:
for p,title in zip([unrestricted, unrestricted_light_unrestricted, intermitent_rest],
                   ['no restrictions', 'early end to restrictions', 'intermittent restrictions']):
    data, actions = run_simulation(p)
    print(p.__name__)
    df = make_SIR_graph(data)
    actions_colors = list(map(color_map.get,  actions))
    make_beds_graph(df, actions_colors, 7, title, color_map)

  0%|                                                                                          | 0/280 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'shape'